# Optimization

This section includes code for optimizing the neural network, including various optimizers. Functions include:

- gradient: Estimates the gradient of a function at a certain point using the central difference method, which is the most accurate one that doesn't get into complications of imaginary numbers
- hessian: Estimates the Hessian using the same technique, populating the Hessian column by column using the gradient
- derivative, initial_bracker, bisection, minimize, line_search: All used to build up the implentation of line search, which is a technique for finding the optimal step size along a certain direction (usually the gradient). While this is probably a bad idea for DL, I wanted to see just how bad of an idea it is going to be.
- gradient_descent, conjugate_descent, adam, newton's method: An assortment of optimization methods to try with deep learning

Here are some hypotheses for how optimization algorithms will behave. I am writing those before seeing how the network will perform, and I want to see if they turn out to be correct:
- adam will perform the best, although gradient descent and conjugate gradient might not be much worse.
- Newton's method will fail because it relies on having a good quadratic estimation of the function. The problem is that when you use mini-batches, the loss function evalutions become noisy, and both the quadratic estimation and the hessian can be quite wrong, so it might even fail to converge.
- Line search will fail for a similar reason, the noisy loss function evalutation. However, I don't think it will be much worse because going slighly too far along the gradient should not be very bad.

Note: I used the optimization code that I wrote for CS164, but I had to heavily modify the interface, so I did some significant work for this section.

In [205]:
import numpy as np
import matplotlib.pyplot as plt


# We'll start by defining a bunch of functions that will form building blocks for our algorithms

def gradient(f, x, h=1e-5):
    """
    Estimates the value of the gradient of a function at a point x using the central difference method
    """
    grad = []
    forward = np.copy(x)
    forward = forward + h/2
    backward = np.copy(x)
    backward = backward + h/2
    grad = f(forward) - f(backward)/h
    return grad

def derivative(f, x, h=1e-5):
    """
    Simplified version of the gradient function for univariate functions
    """
    return np.array((f(x + h/2) - f(x - h/2))/h)

def hessian(f, x, h=1e-5):
    """
    Estimates the value of the hessian of a function at a point x using the central difference method
    """
    n = len(x)
    hess = np.zeros((n,n))
    for i in range(n):
        forward = np.copy(x)
        forward[i] += h/2
        backward = np.copy(x)
        backward[i] -= h/2
        hess[:,i] = (gradient(f,forward) - gradient(f,backward))/h
    return hess

def initial_bracket(f,x=0,s=1e-2,k=2):
    """
    Finds an initial bracket for function f, using x as the starting point, 
    s as the initial step size and k as the step size expansion factor
    """
    a, ya = x, f(x)
    b, yb = a + s, f(a + s)
    if yb > ya:
        a,b = b,a
        ya, yb = yb, ya
        s = -s
    while True:
        c, yc = b + s, f(b + s)
        if yc > yb:
            return (a,c) if a<c else (c,a)
        a, ya, b, yb = b, yb, c, yc
        s = s*k
        
def bisection(f,a,b,tol=1e-4):
    """ Decreases a bracket [a,b] of the function f by bisection until the width is smaller than the tolerance"""
    if a > b:
        a,b = b,a
    ya, yb = derivative(f,a), derivative(f,b)
    if ya == 0:
        b = a
    if yb == 0:
        a = b
    
    while b - a > tol:
        x = (a + b)/2
        y = derivative(f,x)
        if x == 0:
            a,b = x,x
        elif y*ya > 0: # We can replicate matching the sign by multipling the numbers and checking the resulting sign
            a = x
        else:
            b = x
    return (a,b)

def minimize(f,a,b, bracketing_method=bisection):
    """Minimizes a function within an [a,b] interval, by using a bracketing method  to decrease the bracket, 
    then taking the middle of the interval. Basically just a thin wrapper over the bracketing method."""
    a,b = bracketing_method(f,a,b)
    return (a + b)/2

def line_search(f,x,d):
    """"""
    def obj(step):
        return f(x + d*step)
    a,b = initial_bracket(obj)
    step = minimize(obj, a,b)
    return x + step*d

class GradientDescent():
    def __init__(self, input_dim, output_dim):
        pass
    def update(self, x, g, lr):
        """
        x - vector of weights and bias
        f - weights and bias function
        g - gradient of that function"""
        return x - lr*g
        

def conjugate_descent(f, x, lr=0.01):
    """
    f - Function
    x - initial location for search
    lr - learning rate, a.k.a. step size
    """
    if verbose:
        print(new_x)
    x = new_x
    i += 1
    return points

class nesterov():
    def __init(self, decay=0.4):
        self.decay = decay

def nesterov_momentum(f, x, lr=1e-4, decay=0.4, verbose=False):
    """
    f - Function
    x - initial location for search
    lr - learning rate, a.k.a. step size
    tol - tolerance of stopping condition, minimum difference required to proceed.
    max_iter - Maximum iterations before stopping
    decay - The rate at which momentum decays
    """
    g = 0
    v = 0
    points = np.array([x]) # Keeps track of progress
    for i in range(max_iter):
        g = gradient(f,x + (1 - decay)*v)
        v = (1 - decay)*v - lr*g
        new_x = x + v
        if verbose:
            print(new_x)
        points = np.append(points, np.array([new_x]), axis=0)
        if f(new_x) > f(x) - tol:
        #if np.linalg.norm(new_x - x) < tol:
            break
        x = new_x
    return points

# Default parameters are taken from Keras
class adam():
    """
    Note: """
    def __init__(self, input_dim, output_dim, beta1=0.9,beta2=0.999, epsilon=1e-8):
        self.dim = (input_dim, output_dim)
        self.i = 0
        self.v = np.zeros(self.dim)
        self.s = np.zeros(self.dim)
        self.epsilon = epsilon
        self.beta1 = beta1
        self.beta2 = beta2
    def update(self, x, g, lr):
        """
        x - vector of weights and bias
        f - weights and bias function
        df - gradient of that function"""
        self.v = self.beta1*self.v + (1 - self.beta1)*g
        self.s = self.beta2*self.s + (1 - self.beta2)*np.dot(g,g.T)
        # Correct those quantities
        v_hat = self.v/(1 - beta1**i)
        s_hat = self.s/(1 - beta2**i)
        new_x = x - ((lr*v_hat)/(epsilon + np.sqrt(s_hat)))
        self.i += 1
        return new_x
        

def newtons_method(f, x, tol=1e-4):
    i=0
    points = np.array([x])
    while True:
        grad = gradient(f,x)
        hess = hessian(f,x)
        new_x = x - np.matmul(np.linalg.inv(hess),grad)
        points = np.append(points, np.array([new_x]), axis=0)
        if np.linalg.norm(new_x - x) < tol:
            break
        x = new_x
        i += 1
    print(points)
    return points

# The Dataset

I will be testing my code on a regression problem, namely the [California Housing dataset](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset) from sklearn. In this dataset, we are trying to predict the median house cost in districts in California. This toy dataset will serve as a test of my implementation. I will also train benchmark linear and random forest models for comparison. I don't expect my network to do better than those (deep learning is surprisingly ineffective on tabular data, even advanced and tuned architectures usually perform no better than a GBM or a random forest.)

In [206]:
from tensorflow import keras
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Flatten
x_train = x_train.reshape(60000,-1)
x_test = x_test.reshape(10000,-1)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# The Neural Network
The neural network is a dense connected network with 2 hidden layers and a variable number of neurons in each of them. The weights are initialized with He initialization.

In [305]:
class Layer():
    def __init__(self, inputs, outputs, activation, activation_prime, Optimizer=GradientDescent):
        self.weights = np.random.normal(0,np.sqrt(2/inputs),size=(inputs,outputs)) 
        self.bias = np.random.normal(0,np.sqrt(2/inputs),size=(1,outputs))
        self.activation = activation
        self.activation_prime = activation_prime
        self.optimizer = Optimizer(inputs, outputs)
    def feedforward(self, x):
        self.input = x
        self.values = np.dot(x, self.weights) + self.bias
        self.out = self.activation(self.values)
        return self.out
    def backprop(self, grad, lr):
        # Activation gradient
        print("Gradient:", grad)
        activation_grad = self.activation_prime(self.values)*grad
        print("Activation Gradient:", activation_grad)
        input_grad = np.dot(activation_grad,self.weights.T)
        print("Input Gradient:", input_grad)
        weights_grad = np.dot(self.input.T, activation_grad)
        print("Weights Gradient:", weights_grad)
        self.weights = self.optimizer.update(self.weights, weights_grad, lr)
        self.bias = self.optimizer.update(self.bias, np.mean(activation_grad, axis=0), lr)
        print("Weights:", self.weights)
        print("Biases:", self.bias)
        return input_grad

In [306]:
import numpy as np
def sigmoid(x):
    return 1/(1 + np.power(np.e,-x))

def relu(x):
    return np.maximum(np.zeros(x.shape),x)

def grad_relu(x):
    return x > 0

def mse(true, predicted):
    return np.mean((true - predicted)**2)

def grad_mse(true, predicted):
    return 2*(true - predicted)

def categorical_crossentropy_logit(true, predicted):
    """Both true and predicted are a matrix"""
    return np.mean(-true + np.log(np.sum(np.exp(predicted), axis=0)))

def categorical_crossentropy_logit_grad(true, predicted):
    """A wrapper that allows us to use the cross-entropy function, which requires two inputs
    but differentiate it only with respect to the predicted values"""
    return -true + softmax(predicted)

def categorical_crossentropy(true, predicted):
    """Both true and predicted are a matrix"""
    return np.mean(-np.sum(true*np.log(predicted),axis=0))

def categorical_crossentropy_grad(true, predicted):
    """A wrapper that allows us to use the cross-entropy function, which requires two inputs
    but differentiate it only with respect to the predicted values"""
    return -(true*np.log(predicted) + (1 - true)*np.log(1 - predicted))

def softmax(x):
    e = np.exp(x)
    return e / np.sum(e, axis=1).reshape(-1,1)

def unity(x):
    return x

def ones(x):
    return np.ones(x.shape)
class NeuralNetwork:
    def __init__(self, layers, batch_size=64, lr=0.01, loss = categorical_crossentropy_logit, 
                 loss_grad = categorical_crossentropy_logit_grad):
        # He initialization for hidden layer
        self.layers = layers
        self.batch_size = batch_size
        self.i = 0 # Index for batching
        self.lr = lr # Learning rate
        self.loss = loss
        self.epoch = 0
        self.loss_grad = loss_grad

    def feedforward(self, x):
        out = x
        for layer in self.layers:
            out = layer.feedforward(out)
        return out

    def backprop(self, y, output):
        print("Loss:", self.loss(y, output))
        grad = self.loss_grad(y, output)
        for layer in reversed(self.layers):
            grad = layer.backprop(grad, self.lr)
        
    def batch_descent(self, x, y):
        # Key assumption for this
        datapoints = x.shape[0]
        batch_x = x[self.i:min(self.i + self.batch_size, x.shape[0])]
        batch_y = y[self.i:min(self.i + self.batch_size, y.shape[0])]
        self.i = (self.i + self.batch_size)
        wrap = self.i // datapoints # Flag for whether the batching index exceeded the dataset bounds
        self.epoch += wrap
        self.i =  self.i % datapoints
        # Deal with what happens when we need to "wrap around" the dataset.
        if wrap:
            batch_x = np.concatenate([batch_x, x[0:self.i]], axis=0)
            batch_y = np.concatenate([batch_y, y[0:self.i]], axis=0)
        output = self.feedforward(batch_x)
        self.backprop(batch_y, output)
        
    def fit(self, x, y, epochs=10):
        while self.epoch < epochs:
            self.batch_descent(x,y)



In [307]:
layers = [Layer(x_train.shape[1], 20, relu, grad_relu), Layer(20, 20, relu, grad_relu), Layer(20, 10,unity, ones)]
nn = NeuralNetwork(layers, lr=1e-3)
nn.fit(x_train, y_train)

Loss: 4.434203953890561
Gradient: [[ 0.13082131  0.17824939  0.15912209  0.0755743   0.10504293 -0.95894346
   0.07026001  0.04164849  0.15194149  0.04628343]
 [-0.96457531  0.25593196  0.15649853  0.09364394  0.08786029  0.04218664
   0.08165293  0.09671096  0.08823961  0.06185044]
 [ 0.0708435   0.17788932  0.17652843  0.07306349 -0.88650388  0.0401008
   0.09223382  0.07942878  0.09252272  0.08389302]
 [ 0.08148371 -0.77300792  0.24066016  0.07539951  0.07015827  0.01920692
   0.06341473  0.08872122  0.06359702  0.07036639]
 [ 0.05734298  0.14641742  0.16925947  0.11189421  0.06878204  0.03684459
   0.05481824  0.12362292  0.10279385 -0.87177571]
 [ 0.09095421  0.16050247 -0.7973783   0.07743319  0.08421987  0.03620469
   0.06718291  0.09934773  0.09675086  0.08478236]
 [ 0.06032871 -0.78812896  0.19670057  0.08407545  0.0781269   0.03615707
   0.06901039  0.06179892  0.12839604  0.07353489]
 [ 0.13145169  0.11911874  0.14048412 -0.88447896  0.1216661   0.06830993
   0.04097613  0.0

Weights Gradient: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Weights: [[ 0.01591669  0.02695847  0.00769833 ...  0.04136653 -0.0021478
   0.10384388]
 [ 0.03033902 -0.04929204 -0.0026987  ...  0.01285809 -0.0673914
   0.01265783]
 [ 0.00077553  0.03799287  0.01360043 ...  0.04315526  0.02003682
   0.04036365]
 ...
 [-0.03674979 -0.11362903 -0.08500255 ... -0.0052761  -0.09798742
   0.0689229 ]
 [ 0.0335163   0.06596634  0.02689076 ... -0.03691825  0.02327649
  -0.05254951]
 [ 0.07605543 -0.05334682  0.05188054 ... -0.03599054  0.00375125
   0.00629528]]
Biases: [[ 0.02593209 -0.06079585  0.03372283  0.00304863 -0.0202456  -0.01194417
   0.0395542   0.09117606 -0.0448977  -0.08643314 -0.01797434 -0.04152607
   0.03109499  0.10424669 -0.11200654 -0.06286643 -0.00866286 -0.00905976
  -0.00520452  0.01354634]]
Loss: 4.5434551606981985
Gradient: [[ 0.09737469 -0.79007398  0.16372

Activation Gradient: [[ 0.12146306  0.1705439   0.12150804  0.06455816  0.10825491  0.03655999
  -0.84987227  0.06451946  0.10648288  0.05598187]
 [ 0.06863787  0.21602812  0.10005229  0.10580692 -0.89250845  0.02896499
   0.08662975  0.12569504  0.08610813  0.07458535]
 [ 0.09306132  0.19939491  0.13201802  0.09479039  0.07545251  0.03531593
   0.07436029 -0.88935497  0.09295476  0.09200686]
 [ 0.16623289  0.10860643  0.13293955  0.06155819  0.08323683 -0.9409646
   0.06929541  0.1350948   0.10723703  0.07676348]
 [-0.9141131   0.21929501  0.14501733  0.06556119  0.09802433  0.03302222
   0.12821866  0.08969229  0.0737978   0.06148428]
 [ 0.14495452  0.15697986  0.13237917  0.06559422  0.09168742  0.04270308
  -0.86571193  0.08909534  0.08787512  0.05444319]
 [ 0.11512944  0.13521631 -0.80600133  0.07888104  0.09336358  0.04628288
   0.07587535  0.09514519  0.09800759  0.06809996]
 [ 0.09246863  0.12025769  0.11315709  0.07999209  0.05628041  0.04291488
   0.05030862 -0.73500944  0.08

Activation Gradient: [[ 0.09632623  0.11482721  0.11632641  0.03654711  0.07423287  0.04759646
   0.0384888  -0.66829614  0.06172428  0.08222676]
 [ 0.07547903  0.05327151 -0.76649163  0.18496343  0.0717212   0.05667246
   0.03721069  0.07098149  0.14971544  0.06647637]
 [ 0.10628572  0.09226762  0.15512138  0.10701073  0.07241672  0.05463409
   0.05060993  0.14666445 -0.88637956  0.10136893]
 [ 0.06521579  0.10184052  0.15737899  0.07960337  0.04609916  0.03066733
   0.03967371 -0.68180814  0.07434101  0.08698827]
 [ 0.04612853  0.25639904  0.14528482  0.09159139  0.08165025  0.02830318
  -0.90882653  0.08799613  0.08539949  0.0860737 ]
 [ 0.04587381  0.13351834  0.08598943  0.12010262  0.13578086  0.03933219
   0.06122628  0.23854326  0.0617773  -0.92214409]
 [ 0.12544182  0.05731591 -0.6610694   0.0768998   0.03817697  0.03850258
   0.03305272  0.11770686  0.11491789  0.05905485]
 [ 0.17767593  0.05517733 -0.80632858  0.09740758  0.06968308  0.06200745
   0.05272209  0.08879946  0.1

Activation Gradient: [[ 0.08627139  0.12157473  0.08610029  0.15905274 -0.91671398  0.07269063
   0.04910254  0.11135747  0.10527872  0.12528546]
 [-0.61252929  0.02148572  0.10720889  0.10554091  0.04842056  0.13551407
   0.02913832  0.07307736  0.06969763  0.02244584]
 [ 0.14552835  0.11117166  0.10610365  0.08160239  0.11162925  0.04865785
  -0.86006316  0.06966253  0.12229005  0.06341742]
 [ 0.07442291  0.12064591  0.16598291  0.04213786  0.08718777 -0.92043833
   0.04116562  0.25699988  0.08047047  0.051425  ]
 [ 0.08023383  0.08328816  0.11687731  0.14432825  0.06007651  0.07821372
   0.05788211  0.09103398  0.13153513 -0.84346901]
 [ 0.07373152  0.08523303  0.11290975 -0.73886413  0.06724528  0.09847035
   0.01580875  0.06238912  0.16421454  0.05886179]
 [-0.68294528  0.0311648   0.09295462  0.07919357  0.05592398  0.08562273
   0.07691892  0.0924196   0.12914721  0.03959984]
 [ 0.06135994  0.18352641  0.12163741  0.1423005   0.07377312  0.05788876
   0.04713126  0.08036769  0.1

...

  -0.33216205  0.13270147]]
Biases: [[ 0.03053692 -0.1312836   0.0125871  -0.46430836  0.24929269 -0.06121859
   0.46801868 -0.19944621  0.51205226  0.51296677  0.53997647  0.05347081
   0.20291657 -0.34666841 -0.29298746 -0.26417426  0.4613842   0.33421188
  -0.11478408 -0.30533898]]
Gradient: [[ 1.54245191e-01  2.12465646e-01 -3.38719240e-01 ... -1.06774553e-01
   1.74949966e-01  4.81514836e-02]
 [ 1.23727589e-01  1.43733924e-01 -7.81009153e-02 ... -7.65191880e-02
   2.07464562e-01  3.35670821e-01]
 [ 4.73359664e-02  3.02720090e-02 -1.53610931e-02 ... -3.87758400e-02
   1.24351009e-02  1.92713766e-02]
 ...
 [-8.22327367e-02  3.53776219e-02 -1.32243090e-01 ...  4.47430977e-02
   1.84325960e-01 -1.09312482e-02]
 [-4.30224199e-03  9.30314282e-02  6.26727892e-03 ... -3.39904589e-02
   1.26291349e-01  1.55305268e-01]
 [-5.08973557e-04  4.09939109e-03  8.94727538e-03 ... -6.05208910e-03
   4.78213354e-03 -2.01416158e-04]]
Activation Gradient: [[ 0.15424519  0.         -0.33871924 ... -0.10

   0.00629528]]
Biases: [[ 0.02658943 -0.06070646  0.0336196   0.00349472 -0.02002764 -0.01070151
   0.03966828  0.09221765 -0.0449879  -0.08647284 -0.01789651 -0.04158627
   0.03187851  0.10611848 -0.1117666  -0.06209679 -0.00844701 -0.00857556
  -0.00540705  0.01353841]]
Loss: 7.173837743961627
Gradient: [[ 1.23026431e-03  1.99999675e-02  3.63570761e-02 -2.06686787e-01
   9.93652721e-03  4.39843951e-02  5.92339229e-04  7.02946853e-03
   6.98609202e-02  1.76958284e-02]
 [ 3.22593737e-04  6.72571980e-04  3.16367651e-04  8.69717817e-04
   1.92052981e-02  6.39890811e-03  1.24901208e-04 -2.03725180e-01
   1.28912061e-03  1.74525701e-01]
 [ 1.38651047e-02  7.62619396e-04  9.73298760e-03  1.21201223e-02
   1.65552841e-02  2.80358666e-02  1.26472776e-03 -2.22782619e-01
   1.10245929e-02  1.29421314e-01]
 [ 1.74601716e-02  7.81911686e-03 -1.95698916e-01  1.75619333e-03
   2.94195484e-03  4.87200643e-03  1.09514567e-01  3.02069196e-02
   1.95912515e-02  1.53673477e-03]
 [-9.65929433e-02  1.103

  -1.52223634e+00  3.07245804e+00]]
Weights: [[ 0.22086778 -0.51246321  0.23303232  0.04242486 -0.16698439 -0.15557571
  -0.09902744  0.28784307  0.36652448 -0.0821261 ]
 [-0.05529237 -0.12869498  0.39890626 -0.48003856  0.83715934 -0.30913165
  -0.26952908  0.39626082 -0.12012787 -0.16414462]
 [-0.13198852 -0.35427311 -0.18606191  0.35082824  0.04022723  0.60767254
  -0.0868211  -0.09660924 -0.12750627  0.46539455]
 [ 0.03470806  0.19895246 -0.25294755  0.44403774  0.16212689 -0.53019189
   0.09068306  0.09699032 -0.36009094 -0.56340634]
 [-0.85208312  0.3179646   0.82515532  0.30908983  0.06405769 -0.0918752
  -0.19642228 -0.19680071  0.44178172 -0.21121394]
 [-0.39792559  0.09702964 -0.59471173  0.18715627 -0.31973462  0.23566618
  -0.57585051 -0.18600188  0.08202455  0.06438975]
 [ 0.01387298  0.99688012  0.32344683  0.02861497  0.38508431 -0.0942073
   0.57741027 -0.47247394  0.11282504 -0.06498411]
 [ 0.08048601 -0.14943746 -0.0701629  -0.42938597  0.09842862  0.13217032
  -0.411

Loss: 7.993595741244974
Gradient: [[ 9.91745158e-05  4.08725255e-04  9.09039453e-03  1.99291377e-03
   1.04285310e-03  3.92583045e-03  9.70019563e-06 -1.06675402e-01
   4.70068337e-03  8.54051264e-02]
 [ 2.51639354e-02  1.49702055e-04  2.89551375e-02  1.24890345e-04
   2.47079485e-02  2.75084336e-02 -1.32144188e-01  4.22135031e-05
   2.47907790e-02  7.01147710e-04]
 [ 1.34423875e-05  3.43571607e-05  1.07067567e-03 -3.95267734e-02
   1.16373255e-04  2.64650766e-02  1.08025225e-06  4.37336582e-05
   1.16387315e-02  1.43302895e-04]
 [ 3.44858923e-04  3.39299119e-06  2.30960878e-03  1.80710878e-03
   9.22187323e-04  2.13245717e-01  1.32918971e-05  6.62881673e-06
  -2.20340945e-01  1.68815097e-03]
 [ 6.58123011e-05 -9.25912396e-01  2.67207655e-01  4.95871093e-02
   1.92999757e-02  1.60971992e-02  8.31953938e-04  6.38239157e-04
   5.61756105e-01  1.04283468e-02]
 [ 5.59963214e-04  4.38361220e-04  1.10900456e-02  8.30644446e-02
   7.15793861e-04  2.75881457e-01  1.48994058e-04  1.88984333e-03

KeyboardInterrupt: 

In [290]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)